<b>Data Cleaning</b>

In [1]:
import pandas as pd
import requests
import numpy as np

In [2]:
pd.options.display.max_colwidth=300

In [3]:
surveyDF=pd.read_csv("survey_results_public.csv")
surveyDF

,ResponseId,MainBranch,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,LearnCodeCoursesCert,YearsCode,...,TimeSearching,TimeAnswering,Onboarding,ProfessionalTech,TrueFalse_1,TrueFalse_2,TrueFalse_3,SurveyLength,SurveyEase,ConvertedCompYearly
0,1,None of these,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,I am a developer by profession,"Employed, full-time",Fully remote,Hobby;Contribute to open-source projects,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Too long,Difficult,NaN
2,3,"I am not primarily a developer, but I write code sometimes as part of my work","Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Books / Physical media;Friend or family member;Other online resources (e.g., videos, blogs, forum);School (i.e., University, College, etc)",Technical documentation;Blogs;Programming Games;Written Tutorials;Stack Overflow,NaN,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Neither easy nor difficult,40205.0
3,4,I am a developer by profession,"Employed, full-time",Fully remote,I don’t code outside of work,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Books / Physical media;School (i.e., University, College, etc)",NaN,NaN,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Easy,215232.0
4,5,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Other online resources (e.g., videos, blogs, forum);School (i.e., University, College, etc);On the job training","Technical documentation;Blogs;Stack Overflow;Online books;Video-based Online Courses;Online challenges (e.g., daily or weekly coding challenges)",NaN,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Too long,Easy,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73263,73264,I am a developer by profession,"Employed, full-time",Fully remote,Freelance/contract work,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Books / Physical media;Other online resources (e.g., videos, blogs, forum);On the job training;Online Courses or Certification",Technical documentation;Blogs;Written Tutorials;Stack Overflow;Video-based Online Courses;How-to videos,Udemy,8,...,30-60 minutes a day,Less than 15 minutes a day,Just right,DevOps function;Microservices;Developer portal or other central places to find tools/services;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools,Yes,Yes,Yes,Too long,Easy,NaN
73264,73265,I am a developer by profession,"Employed, full-time",Full in-person,Hobby,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Other online resources (e.g., videos, blogs, forum);School (i.e., University, College, etc);On the job training;Online Courses or Certification",Technical documentation;Blogs;Written Tutorials;Stack Overflow;Video-based Online Courses;How-to videos,Coursera;Udemy;Udacity,6,...,15-30 minutes a day,60-120 minutes a day,Very long,None of these,No,Yes,Yes,Too long,Easy,NaN
73265,73266,"I am not primarily a developer, but I write code sometimes as part of my work","Employed, full-time","Hybrid (some remote, some in-person)",Hobby;School or academic work,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Books / Physical media;Other online resources (e.g., videos, blogs, forum);School (i.e., University, College, etc);Online Courses or Certification",Technical documentation;Programming Games;Stack Overflow;Online books;Video-based Online Courses;How-to videos;Written-based Online Courses;Coding sessions (live or recorded);Certification videos,Udemy;Codecademy;Pluralsight;edX,42,...,30-60 minutes a day,60-120 minutes a day,Just right,None of these,No,No,No,Appropriate in length,Easy,NaN
73266,73267,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;O

In [4]:
surveyDF.drop(['ResponseId','LearnCode','LearnCodeOnline','LearnCodeCoursesCert','OrgSize','VersionControlSystem','SurveyEase',
               'SurveyLength','TBranch','ICorPM','Knowledge_1','Knowledge_1','Knowledge_2','Knowledge_3','Knowledge_4','Knowledge_5',
               'Knowledge_6','Knowledge_7','CodingActivities','YearsCode','PurchaseInfluence','BuyNewTool','VersionControlSystem',
               'VCInteraction','VCHostingPersonal use','VCHostingProfessional use','OfficeStackAsyncWantToWorkWith',
               'OfficeStackSyncHaveWorkedWith','Blockchain','SOAccount','SOPartFreq','Frequency_1','Frequency_2',
               'Frequency_3','TimeSearching','TimeAnswering','TrueFalse_1','TrueFalse_2','TrueFalse_3','ConvertedCompYearly',
               'Accessibility','NEWSOSites','SOVisitFreq','SOComm','Onboarding','OfficeStackSyncWantToWorkWith','LanguageWantToWorkWith',
               'WebframeWantToWorkWith','DatabaseWantToWorkWith','NEWCollabToolsWantToWorkWith','PlatformWantToWorkWith',
               'MiscTechWantToWorkWith','ToolsTechWantToWorkWith'],
              axis=1,inplace=True)
surveyDF.head(10)

,MainBranch,Employment,RemoteWork,EdLevel,YearsCodePro,DevType,Country,Currency,CompTotal,CompFreq,...,OpSysPersonal use,OfficeStackAsyncHaveWorkedWith,Age,Gender,Trans,Sexuality,Ethnicity,MentalHealth,WorkExp,ProfessionalTech
0,None of these,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,I am a developer by profession,"Employed, full-time",Fully remote,NaN,NaN,NaN,Canada,CAD\tCanadian dollar,NaN,NaN,...,Windows Subsystem for Linux (WSL),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"I am not primarily a developer, but I write code sometimes as part of my work","Employed, full-time","Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5,"Data scientist or machine learning specialist;Developer, front-end;Engineer, data;Engineer, site reliability",United Kingdom of Great Britain and Northern Ireland,GBP\tPound sterling,32000.0,Yearly,...,Windows,NaN,25-34 years old,Man,No,Bisexual,White,"I have a mood or emotional disorder (e.g., depression, bipolar disorder, etc.);I have an anxiety disorder",NaN,NaN
3,I am a developer by profession,"Employed, full-time",Fully remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",17,"Developer, full-stack",Israel,ILS\tIsraeli new shekel,60000.0,Monthly,...,Windows,Jira Work Management;Trello,35-44 years old,Man,No,Straight / Heterosexual,White,None of the above,NaN,NaN
4,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",3,"Developer, front-end;Developer, full-stack;Developer, back-end;Developer, desktop or enterprise applications;Developer, QA or test",United States of America,USD\tUnited States dollar,NaN,NaN,...,macOS;Windows,NaN,25-34 years old,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"I am not primarily a developer, but I write code sometimes as part of my work","Student, full-time",NaN,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",NaN,NaN,Germany,NaN,NaN,NaN,...,macOS,Confluence,25-34 years old,"Or, in your own words:","Or, in your own words:",Prefer to self-describe:,"Or, in your own words:","Or, in your own words:",NaN,NaN
6,I code primarily as a hobby,"Student, part-time",NaN,"Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)",NaN,NaN,India,NaN,NaN,NaN,...,macOS,NaN,Under 18 years old,Man,No,NaN,Indian,None of the above,NaN,NaN
7,I am a developer by profession,"Not employed, but looking for work",NaN,Some college/university study without earning a degree,NaN,"Developer, full-stack;Student",India,NaN,NaN,NaN,...,Windows,NaN,18-24 years old,Man,No,Straight / Heterosexual,Indian,None of the above,NaN,NaN
8,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",6,"Developer, back-end",Netherlands,EUR European Euro,46000.0,Yearly,...,Windows,Confluence;Jira Work Management,25-34 years old,Woman,No,Prefer to self-describe:,European,"Or, in your own words:",6.0,Innersource initiative;DevOps function;Microservices;Developer portal or other central places to find tools/services;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools
9,I am a developer by profession,"Independent contractor, freelancer, or self-employed",Fully remote,Some college/university study without earning a degree,30,"Developer, desktop or enterprise applications;Developer, mobile;Educator",Croatia,HRK\tCroatian kuna,NaN,NaN,...,Windows,NaN,45-54 years old,Woman,No,Straight / Heterosexual,White;European,None of the above,NaN,NaN


In [5]:
surveyDF.dropna(how=("any"),inplace=True)
surveyDF

,MainBranch,Employment,RemoteWork,EdLevel,YearsCodePro,DevType,Country,Currency,CompTotal,CompFreq,...,OpSysPersonal use,OfficeStackAsyncHaveWorkedWith,Age,Gender,Trans,Sexuality,Ethnicity,MentalHealth,WorkExp,ProfessionalTech
11,"I am not primarily a developer, but I write code sometimes as part of my work","Employed, full-time;Independent contractor, freelancer, or self-employed",Fully remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",10,Engineering manager,United States of America,USD\tUnited States dollar,194400.0,Yearly,...,Linux-based;macOS,Confluence;Jira Work Management;Notion;Trello,35-44 years old,Man,No,Straight / Heterosexual,White,"I have a mood or emotional disorder (e.g., depression, bipolar disorder, etc.)",14.0,Innersource initiative;DevOps function;Microservices;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools
12,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",5,"Developer, full-stack",United States of America,USD\tUnited States dollar,65000.0,Yearly,...,Linux-based,Jira Work Management;Trello,25-34 years old,Man,No,Straight / Heterosexual,White,None of the above,5.0,DevOps function;Microservices
14,I am a developer by profession,"Employed, full-time;Independent contractor, freelancer, or self-employed",Fully remote,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5,"Developer, full-stack;Academic researcher;DevOps specialist",United States of America,USD\tUnited States dollar,110000.0,Yearly,...,macOS,Asana;Jira Work Management;Trello,25-34 years old,Man,No,Straight / Heterosexual,White,None of the above,5.0,DevOps function;Continuous integration (CI) and (more often) continuous delivery;Automated testing
21,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",4,"Developer, full-stack",Italy,EUR European Euro,32000.0,Yearly,...,macOS,Confluence;Notion,25-34 years old,Man,No,Straight / Heterosexual,European,None of the above,4.0,None of these
22,I am a developer by profession,"Employed, full-time",Fully remote,Something else,20,"Developer, back-end",Canada,CAD\tCanadian dollar,125000.0,Yearly,...,Windows,Confluence,35-44 years old,Man,No,Straight / Heterosexual,White;North American,None of the above,23.0,DevOps function;Microservices;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73230,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",7,"Developer, back-end;Cloud infrastructure engineer",Australia,AUD\tAustralian dollar,135000.0,Yearly,...,macOS,ClickUp;Confluence;Jira Work Management;Notion;Trello,35-44 years old,Man,No,Straight / Heterosexual,White;European,"I have a concentration and/or memory disorder (e.g., ADHD, etc.)",7.0,DevOps function;Microservices;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools
73246,I am a developer by profession,"Employed, full-time",Fully remote,Some college/university study without earning a degree,5,"Developer, back-end;Developer, desktop or enterprise applications;Database administrator",Russian Federation,RUB\tRussian ruble,150000.0,Monthly,...,Windows,Confluence;Jira Work Management,35-44 years old,Man,No,Straight / Heterosexual,White;European,None of the above,5.0,DevOps function;Microservices;Developer portal or other central places to find tools/services;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools
73251,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",8,"Developer, full-stack",France,EUR European Euro,

In [6]:
newDF=surveyDF.reset_index(drop=True)
newDF

,MainBranch,Employment,RemoteWork,EdLevel,YearsCodePro,DevType,Country,Currency,CompTotal,CompFreq,...,OpSysPersonal use,OfficeStackAsyncHaveWorkedWith,Age,Gender,Trans,Sexuality,Ethnicity,MentalHealth,WorkExp,ProfessionalTech
0,"I am not primarily a developer, but I write code sometimes as part of my work","Employed, full-time;Independent contractor, freelancer, or self-employed",Fully remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",10,Engineering manager,United States of America,USD\tUnited States dollar,194400.0,Yearly,...,Linux-based;macOS,Confluence;Jira Work Management;Notion;Trello,35-44 years old,Man,No,Straight / Heterosexual,White,"I have a mood or emotional disorder (e.g., depression, bipolar disorder, etc.)",14.0,Innersource initiative;DevOps function;Microservices;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools
1,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",5,"Developer, full-stack",United States of America,USD\tUnited States dollar,65000.0,Yearly,...,Linux-based,Jira Work Management;Trello,25-34 years old,Man,No,Straight / Heterosexual,White,None of the above,5.0,DevOps function;Microservices
2,I am a developer by profession,"Employed, full-time;Independent contractor, freelancer, or self-employed",Fully remote,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5,"Developer, full-stack;Academic researcher;DevOps specialist",United States of America,USD\tUnited States dollar,110000.0,Yearly,...,macOS,Asana;Jira Work Management;Trello,25-34 years old,Man,No,Straight / Heterosexual,White,None of the above,5.0,DevOps function;Continuous integration (CI) and (more often) continuous delivery;Automated testing
3,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",4,"Developer, full-stack",Italy,EUR European Euro,32000.0,Yearly,...,macOS,Confluence;Notion,25-34 years old,Man,No,Straight / Heterosexual,European,None of the above,4.0,None of these
4,I am a developer by profession,"Employed, full-time",Fully remote,Something else,20,"Developer, back-end",Canada,CAD\tCanadian dollar,125000.0,Yearly,...,Windows,Confluence,35-44 years old,Man,No,Straight / Heterosexual,White;North American,None of the above,23.0,DevOps function;Microservices;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8006,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",7,"Developer, back-end;Cloud infrastructure engineer",Australia,AUD\tAustralian dollar,135000.0,Yearly,...,macOS,ClickUp;Confluence;Jira Work Management;Notion;Trello,35-44 years old,Man,No,Straight / Heterosexual,White;European,"I have a concentration and/or memory disorder (e.g., ADHD, etc.)",7.0,DevOps function;Microservices;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools
8007,I am a developer by profession,"Employed, full-time",Fully remote,Some college/university study without earning a degree,5,"Developer, back-end;Developer, desktop or enterprise applications;Database administrator",Russian Federation,RUB\tRussian ruble,150000.0,Monthly,...,Windows,Confluence;Jira Work Management,35-44 years old,Man,No,Straight / Heterosexual,White;European,None of the above,5.0,DevOps function;Microservices;Developer portal or other central places to find tools/services;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools
8008,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",8,"Developer, full-stack",France,EUR European Euro,65000.0,

In [7]:
newDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8011 entries, 0 to 8010
Data columns (total 28 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   MainBranch                      8011 non-null   object 
 1   Employment                      8011 non-null   object 
 2   RemoteWork                      8011 non-null   object 
 3   EdLevel                         8011 non-null   object 
 4   YearsCodePro                    8011 non-null   object 
 5   DevType                         8011 non-null   object 
 6   Country                         8011 non-null   object 
 7   Currency                        8011 non-null   object 
 8   CompTotal                       8011 non-null   float64
 9   CompFreq                        8011 non-null   object 
 10  LanguageHaveWorkedWith          8011 non-null   object 
 11  DatabaseHaveWorkedWith          8011 non-null   object 
 12  PlatformHaveWorkedWith          80

In [8]:
newDF["WorkExp"].astype("float")

0       14.0
1        5.0
2        5.0
3        4.0
4       23.0
        ... 
8006     7.0
8007     5.0
8008     8.0
8009     2.0
8010     5.0
Name: WorkExp, Length: 8011, dtype: float64

In [9]:
newDF["YearsCodePro"]=newDF["YearsCodePro"].str.replace("Less than 1 year","1")
newDF

,MainBranch,Employment,RemoteWork,EdLevel,YearsCodePro,DevType,Country,Currency,CompTotal,CompFreq,...,OpSysPersonal use,OfficeStackAsyncHaveWorkedWith,Age,Gender,Trans,Sexuality,Ethnicity,MentalHealth,WorkExp,ProfessionalTech
0,"I am not primarily a developer, but I write code sometimes as part of my work","Employed, full-time;Independent contractor, freelancer, or self-employed",Fully remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",10,Engineering manager,United States of America,USD\tUnited States dollar,194400.0,Yearly,...,Linux-based;macOS,Confluence;Jira Work Management;Notion;Trello,35-44 years old,Man,No,Straight / Heterosexual,White,"I have a mood or emotional disorder (e.g., depression, bipolar disorder, etc.)",14.0,Innersource initiative;DevOps function;Microservices;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools
1,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",5,"Developer, full-stack",United States of America,USD\tUnited States dollar,65000.0,Yearly,...,Linux-based,Jira Work Management;Trello,25-34 years old,Man,No,Straight / Heterosexual,White,None of the above,5.0,DevOps function;Microservices
2,I am a developer by profession,"Employed, full-time;Independent contractor, freelancer, or self-employed",Fully remote,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5,"Developer, full-stack;Academic researcher;DevOps specialist",United States of America,USD\tUnited States dollar,110000.0,Yearly,...,macOS,Asana;Jira Work Management;Trello,25-34 years old,Man,No,Straight / Heterosexual,White,None of the above,5.0,DevOps function;Continuous integration (CI) and (more often) continuous delivery;Automated testing
3,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",4,"Developer, full-stack",Italy,EUR European Euro,32000.0,Yearly,...,macOS,Confluence;Notion,25-34 years old,Man,No,Straight / Heterosexual,European,None of the above,4.0,None of these
4,I am a developer by profession,"Employed, full-time",Fully remote,Something else,20,"Developer, back-end",Canada,CAD\tCanadian dollar,125000.0,Yearly,...,Windows,Confluence,35-44 years old,Man,No,Straight / Heterosexual,White;North American,None of the above,23.0,DevOps function;Microservices;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8006,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",7,"Developer, back-end;Cloud infrastructure engineer",Australia,AUD\tAustralian dollar,135000.0,Yearly,...,macOS,ClickUp;Confluence;Jira Work Management;Notion;Trello,35-44 years old,Man,No,Straight / Heterosexual,White;European,"I have a concentration and/or memory disorder (e.g., ADHD, etc.)",7.0,DevOps function;Microservices;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools
8007,I am a developer by profession,"Employed, full-time",Fully remote,Some college/university study without earning a degree,5,"Developer, back-end;Developer, desktop or enterprise applications;Database administrator",Russian Federation,RUB\tRussian ruble,150000.0,Monthly,...,Windows,Confluence;Jira Work Management,35-44 years old,Man,No,Straight / Heterosexual,White;European,None of the above,5.0,DevOps function;Microservices;Developer portal or other central places to find tools/services;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools
8008,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",8,"Developer, full-stack",France,EUR European Euro,65000.0,

In [10]:
newDF["YearsCodePro"].astype("float")

0       10.0
1        5.0
2        5.0
3        4.0
4       20.0
        ... 
8006     7.0
8007     5.0
8008     8.0
8009     2.0
8010     5.0
Name: YearsCodePro, Length: 8011, dtype: float64

In [11]:
currencies = newDF["Currency"].unique()
currencies

array(['USD\tUnited States dollar', 'EUR European Euro',
       'CAD\tCanadian dollar', 'PLN\tPolish zloty', 'GBP\tPound sterling',
       'UYU\tUruguayan peso', 'SEK\tSwedish krona', 'INR\tIndian rupee',
       'CNY\tChinese Yuan Renminbi', 'BRL\tBrazilian real',
       'VND\tVietnamese dong', 'TND\tTunisian dinar', 'DKK\tDanish krone',
       'CHF\tSwiss franc', 'THB\tThai baht', 'NOK\tNorwegian krone',
       'CZK\tCzech koruna', 'HUF\tHungarian forint', 'HRK\tCroatian kuna',
       'BAM\tBosnia and Herzegovina convertible mark',
       'ILS\tIsraeli new shekel', 'IMP\tManx pound', 'TRY\tTurkish lira',
       'CRC\tCosta Rican colon', 'MAD\tMoroccan dirham',
       'ZAR\tSouth African rand', 'MXN\tMexican peso',
       'RUB\tRussian ruble', 'AUD\tAustralian dollar',
       'IDR\tIndonesian rupiah', 'LBP\tLebanese pound',
       'JPY\tJapanese yen', 'ARS\tArgentine peso', 'PKR\tPakistani rupee',
       'HKD\tHong Kong dollar', 'EGP\tEgyptian pound',
       'NZD\tNew Zealand dollar', 

In [12]:
# get currency codes
currency_codes=[]
for c in currencies:
  currency_codes.append(c[0:3])
  # replace "Currency" column with currency codes
  newDF.loc[newDF["Currency"]==c, "Currency"] = c[0:3]
newDF.head() # check for updated column

,MainBranch,Employment,RemoteWork,EdLevel,YearsCodePro,DevType,Country,Currency,CompTotal,CompFreq,...,OpSysPersonal use,OfficeStackAsyncHaveWorkedWith,Age,Gender,Trans,Sexuality,Ethnicity,MentalHealth,WorkExp,ProfessionalTech
0,"I am not primarily a developer, but I write code sometimes as part of my work","Employed, full-time;Independent contractor, freelancer, or self-employed",Fully remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",10,Engineering manager,United States of America,USD,194400.0,Yearly,...,Linux-based;macOS,Confluence;Jira Work Management;Notion;Trello,35-44 years old,Man,No,Straight / Heterosexual,White,"I have a mood or emotional disorder (e.g., depression, bipolar disorder, etc.)",14.0,Innersource initiative;DevOps function;Microservices;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools
1,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",5,"Developer, full-stack",United States of America,USD,65000.0,Yearly,...,Linux-based,Jira Work Management;Trello,25-34 years old,Man,No,Straight / Heterosexual,White,None of the above,5.0,DevOps function;Microservices
2,I am a developer by profession,"Employed, full-time;Independent contractor, freelancer, or self-employed",Fully remote,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5,"Developer, full-stack;Academic researcher;DevOps specialist",United States of America,USD,110000.0,Yearly,...,macOS,Asana;Jira Work Management;Trello,25-34 years old,Man,No,Straight / Heterosexual,White,None of the above,5.0,DevOps function;Continuous integration (CI) and (more often) continuous delivery;Automated testing
3,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",4,"Developer, full-stack",Italy,EUR,32000.0,Yearly,...,macOS,Confluence;Notion,25-34 years old,Man,No,Straight / Heterosexual,European,None of the above,4.0,None of these
4,I am a developer by profession,"Employed, full-time",Fully remote,Something else,20,"Developer, back-end",Canada,CAD,125000.0,Yearly,...,Windows,Confluence,35-44 years old,Man,No,Straight / Heterosexual,White;North American,None of the above,23.0,DevOps function;Microservices;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools


In [13]:
# get currency exchange rates from May 24, 2022 to match methodology of original survey analysis
url = 'https://api.currencyapi.com/v3/historical?apikey=2dZBW01p3MVzbNmoHXdBpLU7nroSBsA0U9hPU4A2&currencies=&date=2022-05-24'
conversions = requests.get(url).json()
# save conversion rates in a df
conversion_rates_df = pd.DataFrame(conversions['data'].values())
conversion_rates_df

,code,value
0,AED,3.673160
1,AFN,89.502630
2,ALL,112.878030
3,AMD,452.708330
4,ANG,1.803370
...,...,...
148,YER,250.257430
149,ZAR,15.652040
150,ZMK,9001.200000
151,ZMW,17.160400


In [22]:
for c in currency_codes:
  # grab value from conversion rates df
  value = conversion_rates_df["value"].loc[conversion_rates_df["code"]==c].median()
  # print(c, value)
  # add value to newDF
  newDF.loc[newDF["Currency"]==c, "ConversionRates"] = value
newDF.head() # check for updated column

,MainBranch,Employment,RemoteWork,EdLevel,YearsCodePro,DevType,Country,Currency,CompTotal,CompFreq,...,Age,Gender,Trans,Sexuality,Ethnicity,MentalHealth,WorkExp,ProfessionalTech,ConversionRates,ConvertedComp
0,"I am not primarily a developer, but I write code sometimes as part of my work","Employed, full-time;Independent contractor, freelancer, or self-employed",Fully remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",10,Engineering manager,United States of America,USD,194400.0,Yearly,...,35-44 years old,Man,No,Straight / Heterosexual,White,"I have a mood or emotional disorder (e.g., depression, bipolar disorder, etc.)",14.0,Innersource initiative;DevOps function;Microservices;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools,1.00000,194400.00
1,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",5,"Developer, full-stack",United States of America,USD,65000.0,Yearly,...,25-34 years old,Man,No,Straight / Heterosexual,White,None of the above,5.0,DevOps function;Microservices,1.00000,65000.00
2,I am a developer by profession,"Employed, full-time;Independent contractor, freelancer, or self-employed",Fully remote,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5,"Developer, full-stack;Academic researcher;DevOps specialist",United States of America,USD,110000.0,Yearly,...,25-34 years old,Man,No,Straight / Heterosexual,White,None of the above,5.0,DevOps function;Continuous integration (CI) and (more often) continuous delivery;Automated testing,1.00000,110000.00
3,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",4,"Developer, full-stack",Italy,EUR,32000.0,Yearly,...,25-34 years old,Man,No,Straight / Heterosexual,European,None of the above,4.0,None of these,0.93156,34408.60
4,I am a developer by profession,"Employed, full-time",Fully remote,Something else,20,"Developer, back-end",Canada,CAD,125000.0,Yearly,...,35-44 years old,Man,No,Straight / Heterosexual,White;North American,None of the above,23.0,DevOps function;Microservices;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools,1.28233,97656.25
5,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",9,"Developer, back-end;Developer, mobile",Germany,EUR,85000.0,Yearly,...,25-34 years old,Man,No,Straight / Heterosexual,White;European,None of the above,9.0,DevOps function;Microservices;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools,0.93156,91397.85
6,I am a developer by profession,"Employed, full-time;Independent contractor, freelancer, or self-employed",Fully remote,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",22,"Developer, full-stack",Ireland,EUR,130000.0,Monthly,...,45-54 years old,Man,No,Straight / Heterosexual,White,None of the above,22.0,DevOps function;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools,0.93156,139784.95
7,I am a developer by profession,"Independent contractor, freelancer, or self-employed",Fully remote,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5,"Developer, back-end",Poland,PLN,30000.0,Monthly,...,25-34 years old,Man,No,Straight / Heterosexual,European,None of the above,6.0,DevOps function;Developer portal or other central places to find tools/services;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools,4.28291,7009.35
8,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",3,"Developer, back-end",Poland,PLN,14000.0,Monthly,...,25-34 years old,Man,No,Straight / Heterosexual,White;European,"I have a mood or emotional disorder (e.g., depressio

In [23]:
# add column 'ConvertedComp' by dividing CompTotal by ConversionRates
newDF['ConvertedComp'] = (newDF['CompTotal'] / newDF['ConversionRates']).round(decimals=2)
newDF.head() # check for updated column

,MainBranch,Employment,RemoteWork,EdLevel,YearsCodePro,DevType,Country,Currency,CompTotal,CompFreq,...,Age,Gender,Trans,Sexuality,Ethnicity,MentalHealth,WorkExp,ProfessionalTech,ConversionRates,ConvertedComp
0,"I am not primarily a developer, but I write code sometimes as part of my work","Employed, full-time;Independent contractor, freelancer, or self-employed",Fully remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",10,Engineering manager,United States of America,USD,194400.0,Yearly,...,35-44 years old,Man,No,Straight / Heterosexual,White,"I have a mood or emotional disorder (e.g., depression, bipolar disorder, etc.)",14.0,Innersource initiative;DevOps function;Microservices;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools,1.00000,194400.00
1,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",5,"Developer, full-stack",United States of America,USD,65000.0,Yearly,...,25-34 years old,Man,No,Straight / Heterosexual,White,None of the above,5.0,DevOps function;Microservices,1.00000,65000.00
2,I am a developer by profession,"Employed, full-time;Independent contractor, freelancer, or self-employed",Fully remote,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5,"Developer, full-stack;Academic researcher;DevOps specialist",United States of America,USD,110000.0,Yearly,...,25-34 years old,Man,No,Straight / Heterosexual,White,None of the above,5.0,DevOps function;Continuous integration (CI) and (more often) continuous delivery;Automated testing,1.00000,110000.00
3,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",4,"Developer, full-stack",Italy,EUR,32000.0,Yearly,...,25-34 years old,Man,No,Straight / Heterosexual,European,None of the above,4.0,None of these,0.93156,34350.98
4,I am a developer by profession,"Employed, full-time",Fully remote,Something else,20,"Developer, back-end",Canada,CAD,125000.0,Yearly,...,35-44 years old,Man,No,Straight / Heterosexual,White;North American,None of the above,23.0,DevOps function;Microservices;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools,1.28233,97478.81


In [29]:
# calculate yearly compensation assuming 12 working months and 50 working weeks
def yearly_compensation(frequency, amount):
  if frequency=='Monthly':
    return amount*12
  elif frequency=='Weekly':
    return amount*50
  else:
    return amount

newDF['YearlyComp'] = np.vectorize(yearly_compensation)(frequency=newDF['CompFreq'], amount=newDF['ConvertedComp'])
newDF.head() # check for updated column

,MainBranch,Employment,RemoteWork,EdLevel,YearsCodePro,DevType,Country,Currency,CompTotal,CompFreq,...,Gender,Trans,Sexuality,Ethnicity,MentalHealth,WorkExp,ProfessionalTech,ConversionRates,ConvertedComp,YearlyComp
0,"I am not primarily a developer, but I write code sometimes as part of my work","Employed, full-time;Independent contractor, freelancer, or self-employed",Fully remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",10,Engineering manager,United States of America,USD,194400.0,Yearly,...,Man,No,Straight / Heterosexual,White,"I have a mood or emotional disorder (e.g., depression, bipolar disorder, etc.)",14.0,Innersource initiative;DevOps function;Microservices;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools,1.00000,194400.00,194400.00
1,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",5,"Developer, full-stack",United States of America,USD,65000.0,Yearly,...,Man,No,Straight / Heterosexual,White,None of the above,5.0,DevOps function;Microservices,1.00000,65000.00,65000.00
2,I am a developer by profession,"Employed, full-time;Independent contractor, freelancer, or self-employed",Fully remote,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5,"Developer, full-stack;Academic researcher;DevOps specialist",United States of America,USD,110000.0,Yearly,...,Man,No,Straight / Heterosexual,White,None of the above,5.0,DevOps function;Continuous integration (CI) and (more often) continuous delivery;Automated testing,1.00000,110000.00,110000.00
3,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",4,"Developer, full-stack",Italy,EUR,32000.0,Yearly,...,Man,No,Straight / Heterosexual,European,None of the above,4.0,None of these,0.93156,34350.98,34350.98
4,I am a developer by profession,"Employed, full-time",Fully remote,Something else,20,"Developer, back-end",Canada,CAD,125000.0,Yearly,...,Man,No,Straight / Heterosexual,White;North American,None of the above,23.0,DevOps function;Microservices;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools,1.28233,97478.81,97478.81
5,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",9,"Developer, back-end;Developer, mobile",Germany,EUR,85000.0,Yearly,...,Man,No,Straight / Heterosexual,White;European,None of the above,9.0,DevOps function;Microservices;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools,0.93156,91244.79,91244.79
6,I am a developer by profession,"Employed, full-time;Independent contractor, freelancer, or self-employed",Fully remote,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",22,"Developer, full-stack",Ireland,EUR,130000.0,Monthly,...,Man,No,Straight / Heterosexual,White,None of the above,22.0,DevOps function;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools,0.93156,139550.86,1674610.32
7,I am a developer by profession,"Independent contractor, freelancer, or self-employed",Fully remote,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5,"Developer, back-end",Poland,PLN,30000.0,Monthly,...,Man,No,Straight / Heterosexual,European,None of the above,6.0,DevOps function;Developer portal or other central places to find tools/services;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools,4.28291,7004.58,84054.96
8,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",3,"Developer, back-end",Poland,PLN,14000.0,Monthly,...,Man,No,Straight / Heterosexual,White;European,"I have a mood or emotional disorder (e.g., depression, bipolar disorder, etc.)",3.0,DevOps function;Microservices

In [30]:
# drop Currency, CompTotal, CompFreq, ConversionRates, and ConvertedComp - not needed anymore
newDF.drop(['Currency', 'CompTotal', 'CompFreq', 'ConversionRates', 'ConvertedComp'], axis=1, inplace=True)
newDF.head()

,MainBranch,Employment,RemoteWork,EdLevel,YearsCodePro,DevType,Country,LanguageHaveWorkedWith,DatabaseHaveWorkedWith,PlatformHaveWorkedWith,...,OfficeStackAsyncHaveWorkedWith,Age,Gender,Trans,Sexuality,Ethnicity,MentalHealth,WorkExp,ProfessionalTech,YearlyComp
0,"I am not primarily a developer, but I write code sometimes as part of my work","Employed, full-time;Independent contractor, freelancer, or self-employed",Fully remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",10,Engineering manager,United States of America,C#;HTML/CSS;JavaScript;PowerShell;Python;Rust;SQL,Couchbase;CouchDB;Microsoft SQL Server;MongoDB;MySQL;PostgreSQL;Redis,AWS;Microsoft Azure,...,Confluence;Jira Work Management;Notion;Trello,35-44 years old,Man,No,Straight / Heterosexual,White,"I have a mood or emotional disorder (e.g., depression, bipolar disorder, etc.)",14.0,Innersource initiative;DevOps function;Microservices;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools,194400.00
1,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",5,"Developer, full-stack",United States of America,C;HTML/CSS;Rust;SQL;Swift;TypeScript,PostgreSQL,AWS,...,Jira Work Management;Trello,25-34 years old,Man,No,Straight / Heterosexual,White,None of the above,5.0,DevOps function;Microservices,65000.00
2,I am a developer by profession,"Employed, full-time;Independent contractor, freelancer, or self-employed",Fully remote,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5,"Developer, full-stack;Academic researcher;DevOps specialist",United States of America,HTML/CSS;JavaScript;PHP;Python;R;Ruby;Scala,Elasticsearch;MongoDB;Neo4j;PostgreSQL,AWS;DigitalOcean;Heroku,...,Asana;Jira Work Management;Trello,25-34 years old,Man,No,Straight / Heterosexual,White,None of the above,5.0,DevOps function;Continuous integration (CI) and (more often) continuous delivery;Automated testing,110000.00
3,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",4,"Developer, full-stack",Italy,Python;SQL;TypeScript,MySQL;Redis;SQLite,Google Cloud;OVH;VMware,...,Confluence;Notion,25-34 years old,Man,No,Straight / Heterosexual,European,None of the above,4.0,None of these,34350.98
4,I am a developer by profession,"Employed, full-time",Fully remote,Something else,20,"Developer, back-end",Canada,C#;SQL;TypeScript,Microsoft SQL Server;PostgreSQL;Redis,Google Cloud;Microsoft Azure,...,Confluence,35-44 years old,Man,No,Straight / Heterosexual,White;North American,None of the above,23.0,DevOps function;Microservices;Continuous integration (CI) and (more often) continuous delivery;Automated testing;Observability tools,97478.81


In [31]:
newDF.to_csv("cleanedSurvey.csv") # save a clean version of data

<b> Machine Learning </b>

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
import tensorflow as tf

#  Import and read the cleaned data
clean_df = pd.read_csv("cleanedSurvey.csv")
clean_df.head()